# Setup

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
# matplotlib.backend
# matplotlib.validate_backend("GTK3Ag")

In [ ]:
import numpy as np
import pickle
from tqdm.auto import tqdm as progressbar
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import cv2.aruco as aruco
import common as co
from os.path import join
from calib import calibrate_charuco_local, load_board, load_coefficients, save_coefficients

## Load calib

In [ ]:
data_path="/home/freitas/TCC/ender-laser-scanner/pics/v1_raspi/"

In [ ]:
import json

# Calib params
with open("/home/freitas/TCC/ender-laser-scanner/calib_data/calib.pkl", "rb") as f:
    calib = pickle.load(f)
mtx = calib["mtx"]
dist_original = calib["dist"]
dist = None
n = calib["n"]

board, aruco_dict = load_board("board.pkl")

print(board.getChessboardSize())
print(board.getMarkerLength())
print(board.getSquareLength())

In [ ]:
def load(path):
    info = {}
    data = []
    with open(join(path,"points.json")) as f:
        info = json.loads(f.read())

    for fig, point in info.items():
        image =  cv2.cvtColor(cv2.imread(join(path, fig)), cv2.COLOR_BGR2RGB)
        image = cv2.undistort(image, mtx, dist_original, None, mtx)
        data.append( (point, image) )

    return data


arucoParams = aruco.DetectorParameters_create()
def find_charuco_pose(image, mtx, dist):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers( gray, aruco_dict, cameraMatrix=mtx, parameters=arucoParams, distCoeff=dist)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco( markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=mtx, distCoeffs=dist)

    if(resp > 6):
        retval, rvec, tvec	=	cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, mtx, dist, None,None)
    else:
        return None, None
    if not retval:
        print("erro")
    return rvec, tvec

In [ ]:
data = load(data_path+"auto_exp")
coords = []

for point, image in data:
    rvec, tvec = find_charuco_pose(image)
    if rvec is None or tvec is None:
        print("DROP, ", end=" ")
    coords.append((rvec, tvec))

data = load(data_path+"low_exp")

data_bright = load(data_path+"auto_exp")

new_coords = []
new_data = []
good_images = []
for i, ( (point, image), (rvec, tvec) ) in enumerate(zip(data, coords)):
    if rvec is None or tvec is None:
        print(f"dropped {i}")
        continue

    good_images.append(data_bright[i])

    new_data.append((point, image))
    new_coords.append((rvec, tvec))

data = new_data
coords = new_coords

In [ ]:
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None

def init():
    global imgplot, pointsplot
    imgplot = plt.imshow(data[0][1], animated=True)
    pointsplot, = plt.plot(range(data[0][1].shape[0]), "r.", animated=True)
    return pointsplot, imgplot


def laser_points_img_gen(i):
    global imgplot, pointsplot
    # Find board and laser
    point, image =data[i]

    blur = cv2.GaussianBlur(image, ksize=(5,5), sigmaX=1)

    centroids = co.column_centroids(co.red_contrast(blur), mask=co.red_contrast(blur)>50)
    # centroids = co.column_argmaxes(co.red_contrast(image), mask=co.red_contrast(image)>50)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T

    # Update anim
    imgplot.set_data(data[i][1])
    pointsplot.set_data(p_centroids[0], p_centroids[1])
    return pointsplot, imgplot

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=len(data), interval=20, blit=True)

In [ ]:
def ray_plane_intersect_vectorized(img_points, plane, cam_mtx):
	"""
	Calculate 3d-intersection between image rays defined by `img_ponts` and `cam_mtx`
	and a plane defined by the 4-vec `plane`
	@param img_points np.array with shape (3, N), N= number of points, homegeneous coords
	@param plane np.array with shape = (4,)
	@param cam_mtx np.array with shape = (3,3), intrinsic camera parameters
	@returns np.array with shape (4, N) with 3d points
	"""
	assert plane.shape == (4,)
	assert len(img_points.shape) == 2
	assert img_points.shape[0] == 3

	rays = np.linalg.inv(cam_mtx)@img_points
	p = plane[-1]
	n = plane[:3]
	out = (-p/(n@rays)) * rays
	out = np.vstack((out, np.ones((1, out.shape[-1])))) # homogeneous coords
	return out

In [ ]:
def triangulate_pts(image):
    blur = cv2.GaussianBlur(image, ksize=(5,5), sigmaX=1)
    centroids = co.column_centroids(co.red_contrast(blur), mask=co.red_contrast(blur)>50)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T

    pts_3d = ray_plane_intersect_vectorized(p_centroids, n, mtx)
    return pts_3d
def chess_plane(rvec, tvec):
	# Find p
	n_chessboard = np.array([0, 0, 1])
	M,_ = cv2.Rodrigues(rvec)
	n3 = M@n_chessboard
	p = -1 * n3@tvec
	n = np.hstack((n3,p))
	return n

In [ ]:
W = np.eye(4)
Xlist = []
XcList=[]
for (point, image), (rvec, tvec) in zip(data, coords):
    # 3d pts in camera frame
    pts_3d = triangulate_pts(image)
    # Extrinsic params
    rot, jac = cv2.Rodrigues(rvec)
    W[:3, -1] = tvec.flatten()
    W[:3,:3] = rot

    # Concat points
    XcList.append(pts_3d)
    Xlist.append(np.linalg.inv(W)@pts_3d)
X = np.hstack(Xlist)

## Plot

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import importlib
import common as co
importlib.reload(co)

### Gauss Laser

In [ ]:
fig, axs = plt.subplots(2,2, sharey=True, sharex="col")
i = 15
img_1 = data[i][ 1 ]
img_2 = data_bright[i][ 1 ]

axs[0,0].imshow(img_1)
axs[0,0].axvline(500)
axs[0,0].set_title(r"$\tau = 10^*$")
axs[0,1].plot(np.mean(img_1, axis=-1)[:,500],range(720), label = "Intensidade")
axs[0,1].plot(co.red_contrast(img_1)[:,500],range(720), label="Contraste Vermelho (Taubin, 2014)")

red = co.red_contrast(img_1)
red[red<1] = 0
centers = co.column_centroids(red)

c = centers[500]
# axs[0,0].plot(500, c, "r.")
#axs[0,1].axhline(c,  color="r",label="Centróide")

#axs[0,0].axvline(np.argmax(np.mean(img_1, axis=-1)[:,500]))
#axs[0,1].set_xlabel("Valor do Pixel")
axs[0,1].set_ylabel("Linha")
axs[0,1].invert_yaxis()
axs[0,1].legend().set_draggable(True)

axs[1,0].set_title(r"$\tau = 500^*$")

axs[1,0].imshow(img_2)
axs[1,0].axvline(500)
axs[1,1].plot(np.mean(img_2, axis=-1)[:,500], range(720),label = "Intensidade")
axs[1,1].plot(co.red_contrast(img_2)[:,500], range(720), label="Contraste Vermelho (Taubin, 2014)")

print(c)

red = np.mean(img_2, axis=-1)
red[red<150] = 0
centers = co.column_centroids(red)
c = centers[500]
# axs[1,0].plot(500, c, "r.")
#axs[1,1].axhline(c,  color="r", label="Centróide")

print(c)

axs[1,1].set_xlabel("Valor do Pixel")
axs[1,1].set_ylabel("Linha")
axs[1,1].invert_yaxis()
axs[1,1].legend(loc="center right").set_draggable(True)
#axs[1].axvline(np.argmax(co.red_contrast(img_1)[:,500]))

fig.suptitle("Comparação de Tempos de Exposição")


### Profiles

In [ ]:
import matplotlib.ticker as plticker

fig, ax = co.plot3d(proj_type="ortho")
for i, x in enumerate(XcList[3:-3*2:3]):
    #x = x.copy()
    x = x[:,x[2] < 110]
    ax.scatter(x[0], i + x[1] + 24, x[2], label=f"{i}")

#co.set_axes_equal(ax)
ax.set_xlabel("$X_c ~[mm]$")
ax.set_ylabel("$i_p + Y_c ~[mm]$")
ax.set_zlabel("$Z_c ~[mm]$")

ax.invert_zaxis()

#ax.set_ylim(-0.5, i)
#ax.set_zlim(90, 100)
#ax.set_xlim(-60, 0)

#ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
ax.yaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
ax.zaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

ax.legend(loc="upper right", ncol=2, title="Perfil ($i_p$)").set_draggable(True)
plt.title("Visualização 3D dos Perfis")
fig.tight_layout()

In [ ]:
for i, x in enumerate(XcList[3:-3*2:3]):
    x = x[:,x[2] < 110]
    plt.plot(x[0], x[2], "--.", label=f"{i}")

plt.gca().invert_yaxis()

plt.xlabel(r"$X_c~ [mm]$")
plt.ylabel(r"$Z_c~ [mm]$")
ax.legend(loc="best", ncol=2, title="Perfil ($i_p$)")
plt.grid()
plt.title("Projeção dos Perfis em XZ")
plt.legend().set_draggable(True)
plt.tight_layout()

In [ ]:
chosen_pics = slice(3,-3*2,3)

d = data[chosen_pics]
d2 = good_images[chosen_pics]

cols = 2
fig, axs = plt.subplots(len(d)//cols,cols, sharex=True, sharey=True)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

for i, ( ax, (point, image) ) in enumerate( zip(axs.T.flatten(), d) ):

    # Axes
    ax.imshow(d2[i][1])
    ax.axis("off")
    ax.set_title(f"Perfil {i}", color=colors[i])

    blur = cv2.GaussianBlur(image, ksize=(5,5), sigmaX=1)
    centroids = co.column_centroids(co.red_contrast(blur), mask=co.red_contrast(blur)>50)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T

    ax.plot(p_centroids[0], p_centroids[1], "g.", markersize=1)

In [ ]:
fig, ax = co.plot3d()

for x in Xlist:
    ax.scatter(x[0], x[1], x[2])

for (point, image), (rvec, tvec) in zip(data, coords):
    # Axes
    co.plot_axes(ax, np.linalg.inv(co.vec2M(rvec, tvec)))

co.set_axes_equal(ax)

### Axes

In [ ]:
fig, ax = co.plot3d()

for (point, image), (rvec, tvec) in zip(data, coords):
    # Axes
    co.plot_axes(ax, np.linalg.inv(co.vec2M(rvec, tvec)))

# Board
ax.scatter(board.chessboardCorners[:,0], board.chessboardCorners[:,1], board.chessboardCorners[:,2], color="k", label="board")
co.set_axes_equal(ax)
plt.legend()

### Profiles

In [ ]:
print(chess_plane(rvec, tvec))
print(n)

### Reconstruct

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
region = X[2] > -9999
ax.plot_trisurf(X[0][region],X[1][region], X[2][region], color="r")

### Side view

In [ ]:
region = X[2] != 9999
plt.scatter(X[1][region], X[2][region])

# Reconstruct with `cam_to_printer`

In [ ]:
R = np.load("cam_to_printer.npy")
print(R)
p0 = np.array(data[0][0])
print(p0)

cam2printer = np.eye(4)
cam2printer[:3,:3] = R[:3,:3]
print(cam2printer)

In [ ]:
cam2printer

In [ ]:
W = np.eye(4)
Xlist = []
XcList=[]

for (point, image), (rvec, tvec) in zip(data, coords):
    # 3d pts in camera frame
    pts_3d = triangulate_pts(image)

    # Transformation from first point to this one (printer coords)
    B = np.eye(4)
    B[:3,3] = np.array(point) - p0

    # Transformation from the first camera to this one
    A = cam2printer@B@np.linalg.inv(cam2printer)

    # Concat points
    XcList.append(pts_3d)
    Xlist.append(A@pts_3d)
X = np.hstack(Xlist)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#region = X[2] > -9999
ax.plot_trisurf(X[0],X[1], X[2], color="r")

In [ ]:
import matplotlib.ticker as plticker

fig, ax = co.plot3d(proj_type="ortho")
for i, x in enumerate(Xlist):
    #x = np.linalg.inv(cam2printer)@x
    ax.scatter(x[0], x[1], x[2], label=f"{i}")

#ax.set_ylim(-0.5, i)
#ax.set_zlim(90, 100)
#ax.set_xlim(-60, 0)

#ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.yaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.zaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

fig.tight_layout()